# "Easy Street" Trading Bot

We are endeavoring to build a trading algorithm that will outperform a buy and hold strategy for the S&P 500, and evaluate whether or not this program could be used for trading alternative asset clasees that are correlated to the equity markets like crypto.

#### Project Steps
### Establishing a Baseline Performance

In this section, we will establish a baseline performance for the trading algorithm. To do so, we will complete the following steps:

1. Import the S&P 500 historical data into a Pandas DataFrame.

2. Generate trading signals using MACD and RSI technical indicators.

3. Backtest the trading signals to determine the profitability of the algorithm.

4. Split the data into training and testing datasets.

5. Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

6. Review the classification report associated with the `SVC` model predictions. 

7. Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

8. Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

9. Write our conclusions about the performance of the baseline trading algorithm in the `README.md` file that’s associated with our GitHub repository.

#### Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. (You’ll choose the best by comparing the cumulative products of the strategy returns.) To do so, complete the following steps:

1. Tune the training algorithm by adjusting the size of the training dataset. To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing the training window?

> **Hint** To adjust the size of the training dataset, you can use a different `DateOffset` value&mdash;for example, six months. Be aware that changing the size of the training dataset also affects the size of the testing dataset.

2. Tune the trading algorithm by adjusting the SMA input features. Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

3. Choose the set of parameters that best improved the trading algorithm returns. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

#### Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. To do so, complete the following steps:

1. Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

2. Using the original training data as the baseline model, fit another model with the new classifier.

3. Backtest the new model to evaluate its performance. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. Answer the following questions: Did this new model perform better or worse than the provided baseline model? Did this new model perform better or worse than your tuned trading algorithm?

#### Create an Evaluation Report

In the previous sections, you updated your `README.md` file with your conclusions. To accomplish this section, you need to add a summary evaluation report at the end of the `README.md` file. For this report, express your final conclusions and analysis. Support your findings by using the PNG images that you created.


In [17]:
# Imports
import pandas as pd
import numpy as np
import io
import pytz
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import yfinance as yf
import pandas_ta as ta


---

## Step 1: Gather data for the trading algorithm, and setup Dataframes

In this section, we will import our pricing data, and generate trading signals.


In [18]:
# Request historic hourly pricing data via finance.yahoo.com API
df = yf.Ticker('SPY').history(period='1y', interval = "1h")[['Close', 'Open', 'High', 'Volume', 'Low']]
df.index.name='Date'
# Review the DataFrame
df.head()

#note the data pulled from the API is already limited to only the hours the market is open

,Close,Open,High,Volume,Low
Date,,,,,
2021-04-13 09:30:00-04:00,411.950012,411.529999,412.149994,11677340,411.119995
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,5872073,411.720001
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,3257155,411.739990
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,6086220,411.540009
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,7161962,411.940002


In [19]:
#export the S&P data to a .csv file

df.to_csv("../Resources/spy.csv")

In [20]:
# Filter the date index, open, high, low and close columns
signals_df = df.loc[:, ["Close","Open", "High","Low"]]

# Use the pct_change function to generate  returns from close prices
signals_df["actual_returns"] = signals_df["Close"].pct_change()

# Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,Close,Open,High,Low,actual_returns
Date,,,,,
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,411.720001,0.000267
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,411.739990,-0.000243
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,411.540009,0.000024
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,411.940002,0.001767
2021-04-13 14:30:00-04:00,413.359985,412.695007,413.528992,412.662811,0.001604


,Close,Open,High,Low,actual_returns
Date,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-0.000340
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-0.007133
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-0.003261
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,0.002768
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,0.000046


In [21]:
# set the initial_capital to 100000
initial_capital = float(100000)

In [22]:
# visualize the S&P price over the past year

signals_df["Close"].hvplot(title="SPY - S&P 500 ETF: 1 year price history", ylabel="price $/share", figsize = (20,15))



:Curve   [Date]   (Close)

In [23]:
# Visualize the return of the S&P 500 over the past year
((((1+signals_df["actual_returns"]).cumprod())-1)*100).hvplot(title="SPY - S&P 500 ETF: 1 year return", ylabel="percent cumulative return", figsize = (20,15))

:Curve   [Date]   (actual_returns)

## Step 2: Generate trading signals using MACD, Stochastics and RSI

In [24]:
# Calculate MACD values using the pandas_ta library
signals_df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)


# View result
display(signals_df.head())
display(signals_df.tail())

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,
2021-04-13 10:30:00-04:00,412.059998,411.950012,412.200012,411.720001,0.000267,NaN,NaN,NaN
2021-04-13 11:30:00-04:00,411.959991,412.059906,412.209991,411.739990,-0.000243,NaN,NaN,NaN
2021-04-13 12:30:00-04:00,411.970001,411.964996,412.119995,411.540009,0.000024,NaN,NaN,NaN
2021-04-13 13:30:00-04:00,412.698090,411.975006,412.720001,411.940002,0.001767,NaN,NaN,NaN
2021-04-13 14:30:00-04:00,413.359985,412.695007,413.528992,412.662811,0.001604,NaN,NaN,NaN


,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-0.000340,-1.904712,-0.045560,-1.859152
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-0.007133,-2.125682,-0.213224,-1.912458
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-0.003261,-2.388655,-0.380958,-2.007697
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,0.002768,-2.470944,-0.370598,-2.100347
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,0.000046,-2.505660,-0.324251,-2.181409


In [25]:
#Calculate RSI values using the pandas-ta library
signals_df.ta.rsi(close="Close", append=True)

Date
2021-04-13 10:30:00-04:00          NaN
2021-04-13 11:30:00-04:00          NaN
2021-04-13 12:30:00-04:00          NaN
2021-04-13 13:30:00-04:00          NaN
2021-04-13 14:30:00-04:00          NaN
                               ...    
2022-04-12 12:30:00-04:00    39.286301
2022-04-12 13:30:00-04:00    32.802059
2022-04-12 14:30:00-04:00    30.352869
2022-04-12 15:30:00-04:00    34.789679
2022-04-12 16:00:00-04:00    34.863614
Name: RSI_14, Length: 1775, dtype: float64

In [26]:
#Calculate stochastic values using the pandas-ta library
signals_df.ta.stoch(close="Close", append=True)

,STOCHk_14_3_3,STOCHd_14_3_3
Date,,
2021-04-15 09:30:00-04:00,NaN,NaN
2021-04-15 10:30:00-04:00,NaN,NaN
2021-04-15 11:30:00-04:00,91.980541,NaN
2021-04-15 12:30:00-04:00,89.150989,NaN
2021-04-15 13:30:00-04:00,94.443276,91.858269
...,...,...
2022-04-12 12:30:00-04:00,28.591089,30.039237
2022-04-12 13:30:00-04:00,14.954800,25.692469
2022-04-12 14:30:00-04:00,8.822045,17.455978


In [27]:
#drop NaN values

signals_df = signals_df.dropna()

signals_df.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3
Date,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568


In [28]:
# Initialize the new Signal columns
signals_df['MACD_Signal'] = 0.0
signals_df["RSI_Signal"] = 0.0
signals_df["Stoch_Signal"]= 0.0
signals_df["custom_signal"] = 0.0

#MACD Logic:
# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['MACDh_12_26_9'] >= 0), 'MACD_Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['MACDh_12_26_9'] < 0), 'MACD_Signal'] = -1

signals_df["MACD_Entry/Exit"] = signals_df["MACD_Signal"].diff()



#RSI Logic:
# When RSI < 30, buy signal are authorized
signals_df.loc[(signals_df['RSI_14'] < 30), 'RSI_Signal'] = 1

#When RSI > 70 sell signals are authorized
signals_df.loc[(signals_df['RSI_14'] > 70), 'RSI_Signal'] = -1

#when  30 < RSI < 70, neutral signal, do not enter new trades. defult value of 0 will suffice.





#Stochastic Logic:
#calculate the difference between the "k" and "d" stochastics
signals_df["stoch_diff"] = signals_df["STOCHk_14_3_3"] - signals_df["STOCHd_14_3_3"]

#when the "k" crosses below the "d", that is a bearish signal.
signals_df.loc[signals_df["stoch_diff"] < 0, "Stoch_Signal"] = -1

#when the "k" crosses above the "d", that is a bullish signal.
signals_df.loc[signals_df["stoch_diff"]  > 0, "Stoch_Signal"] = 1

signals_df["Stoch_Entry/Exit"] = signals_df["Stoch_Signal"].diff()

signals_df.head(50)

#custom logic:
#combine valid MACD and Stochastic signals to only enter a trade when multiple indicators are in agreement.
signals_df["custom_signal"]=0

#def custom_indicator (indicator_1. indicator_2)
for index, row in signals_df.iterrows():

    if (row["Stoch_Entry/Exit"] == 2 and row["MACD_Entry/Exit"] == 2):
        signals_df.at[index, "custom_signal"] = 2
    elif (row["Stoch_Entry/Exit"] == -2 and row["MACD_Entry/Exit"] == -2):
        signals_df.at[index, "custom_signal"] = -2
    else:
        signals_df.at[index, "custom_signal"] = 0    


# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

C:\Users\nmend\AppData\Local\Temp/ipykernel_26580/2406043902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_df['MACD_Signal'] = 0.0
C:\Users\nmend\AppData\Local\Temp/ipykernel_26580/2406043902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_df["RSI_Signal"] = 0.0
C:\Users\nmend\AppData\Local\Temp/ipykernel_26580/2406043902.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,-1.0,0.0,1.0,0,NaN,3.827184,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0,0.0,-0.903208,0.0


,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-0.000340,-1.904712,-0.045560,-1.859152,39.286301,28.591089,30.039237,-1.0,0.0,-1.0,0,0.0,-1.448148,-2.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-0.007133,-2.125682,-0.213224,-1.912458,32.802059,14.954800,25.692469,-1.0,0.0,-1.0,0,0.0,-10.737669,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-0.003261,-2.388655,-0.380958,-2.007697,30.352869,8.822045,17.455978,-1.0,0.0,-1.0,0,0.0,-8.633933,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,0.002768,-2.470944,-0.370598,-2.100347,34.789679,7.240768,10.339204,-1.0,0.0,-1.0,0,0.0,-3.098436,0.0
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,0.000046,-2.505660,-0.324251,-2.181409,34.863614,13.034849,9.699221,-1.0,0.0,1.0,0,0.0,3.335629,2.0


In [29]:
# export signals_df to csv
#signals_df.to_csv("../Resources/signals_df.csv")

-----------------

## Create the MACD signal Algorithm

In [30]:
#Create a new dataframe for just the MACD signal algorithms

macd_df = signals_df[["Close", "Open", "High", "Low", "MACD_12_26_9", "MACDh_12_26_9", "MACDs_12_26_9", "MACD_Entry/Exit"]].copy()

macd_df.head()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MACD_Entry/Exit
Date,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.647863,-0.549211,1.197073,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,0.487409,-0.567732,1.055140,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,0.169077,-0.708851,0.877928,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,-0.036787,-0.731771,0.694985,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.265860,-0.768676,0.502816,0.0


In [31]:
#check to see how many signals observed in the period
macd_df['MACD_Entry/Exit'].value_counts()

 0.0    1605
 2.0      68
-2.0      68
Name: MACD_Entry/Exit, dtype: int64

In [32]:
# Try a long-only strategy using MACD to time entries/exits

# Initialize trade_type column to track buys and sells
macd_df["side"] = np.nan

# Initialize a cost/proceeds column for recording trade metrics
macd_df["cost/proceeds"] = np.nan

# Initialize share size and accumulated shares
share_size = 100
accumulated_shares = 0

# Initialize variable to hold previous price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a trade at each iteration 
for index, row in macd_df.iterrows():
    
    # initiate a buy on the first trading day
    if previous_price == 0:
        macd_df.loc[index, "side"] = "buy"
        macd_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        macd_df.loc[index, "trade_volume"] = share_size
        macd_df.loc[index, "MACD_Entry/Exit"] = 2
        accumulated_shares += share_size
        macd_df.loc[index, "shares"] = accumulated_shares
    # buy if the current day's price is less than the previous day's price
    elif row["MACD_Entry/Exit"] > 0:
        macd_df.loc[index, "side"] = "buy"
        macd_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        macd_df.loc[index, "trade_volume"] = share_size
        accumulated_shares += share_size
        macd_df.loc[index, "shares"] = accumulated_shares
    # sell if the current day's price is greater than the previous day's price
    elif row["MACD_Entry/Exit"] < 0:
        macd_df.loc[index, "side"] = "sell" 
        macd_df.loc[index, "cost/proceeds"] = (row["Close"] * accumulated_shares)
        macd_df.loc[index, "trade_volume"] = -share_size
        accumulated_shares = 0
        macd_df.loc[index, "shares"] = accumulated_shares
    # hold if the current day's price is equal to the previous day's price
    else:
        macd_df.loc[index, "side"] = "hold"
        macd_df.loc[index, "shares"] = accumulated_shares
    # update the previous_price to the current row's price
    previous_price = row["Close"]
    

# Review the DataFrame
macd_df.tail()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MACD_Entry/Exit,side,cost/proceeds,trade_volume,shares
Date,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-1.904712,-0.045560,-1.859152,0.0,hold,NaN,NaN,0.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-2.125682,-0.213224,-1.912458,0.0,hold,NaN,NaN,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-2.388655,-0.380958,-2.007697,0.0,hold,NaN,NaN,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,-2.470944,-0.370598,-2.100347,0.0,hold,NaN,NaN,0.0
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,-2.505660,-0.324251,-2.181409,0.0,hold,NaN,NaN,0.0


## Visualize the MACD trading algorithm

In [33]:
#graphing positions over time

macd_df.hvplot(y=["Close", "shares"], kind = "line", figsize = (35,20))



:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [34]:
# Visualize entry position relative to close price
entry = macd_df[macd_df["MACD_Entry/Exit"] == 2.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = macd_df[macd_df["MACD_Entry/Exit"] == -2.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = macd_df[["Close"]].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = macd_df[["MACD_12_26_9", "MACDs_12_26_9"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * entry * exit
entry_exit_plot


:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [35]:
# Visualize moving averages
moving_avgs = macd_df[["MACD_12_26_9", "MACDs_12_26_9"]].hvplot(
    ylabel='value',
    width=1000,
    height=400
)
# Visualize Histogram
histogram = macd_df["MACDh_12_26_9"].hvplot(kind ='bar')

#histogram * moving_avgs

In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1)
# price Line
fig.append_trace(
    go.Scatter(
        x=macd_df.index,
        y=macd_df['Open'],
        line=dict(color='#ff9900', width=1),
        name='Open',
        # showlegend=False,
        legendgroup='1',
    ), row=1, col=1
)
# Candlestick chart for pricing
fig.append_trace(
    go.Candlestick(
        x=macd_df.index,
        open=macd_df['Open'],
        high=macd_df['High'],
        low=macd_df['Low'],
        close=macd_df['Close'],
        increasing_line_color='#ff9900',
        decreasing_line_color='black',
        showlegend=False
    ), row=1, col=1
)
# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=macd_df.index,
        y=macd_df['MACD_12_26_9'],
        line=dict(color='#ff9900', width=2),
        name='macd',
        # showlegend=False,
        legendgroup='2',
    ), row=2, col=1
)
# Slow signal (%d)
fig.append_trace(
    go.Scatter(
        x=macd_df.index,
        y=macd_df['MACDs_12_26_9'],
        line=dict(color='#000000', width=2),
        # showlegend=False,
        legendgroup='2',
        name='signal'
    ), row=2, col=1
)
# Colorize the histogram values
colors = np.where(macd_df['MACDh_12_26_9'] < 0, '#000', '#ff9900')
# Plot the histogram
fig.append_trace(
    go.Bar(
        x=macd_df.index,
        y=macd_df['MACDh_12_26_9'],
        name='histogram',
        marker_color=colors,
    ), row=2, col=1
)
# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
# Update options and show plot
fig.update_layout(layout,margin=dict(l=20, r=20, t=20, b=20))
fig.show()

# Backtest the MACD Algorithm

In [37]:
#let's see how much money our system made!

total_profit_loss = macd_df["cost/proceeds"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $1916.88232421875!


In [38]:
macd_df["value"] =  macd_df["trade_volume"] * macd_df["Close"]

In [39]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

macd_trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss']
)



In [40]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0


In [41]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in macd_df.iterrows():
    if row['MACD_Entry/Exit'] == 2:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['trade_volume']
        entry_share_price = row['Close']

    elif row['MACD_Entry/Exit'] == -2:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['trade_volume'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        macd_trade_evaluation_df = macd_trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
macd_trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,SPY,2021-04-21 12:30:00-04:00,2021-04-22 14:30:00-04:00,100.0,414.700989,412.174988,41470.098877,41217.498779,-252.600098
1,SPY,2021-04-23 11:30:00-04:00,2021-04-27 10:30:00-04:00,100.0,416.415009,417.195007,41641.500854,41719.500732,77.999878
2,SPY,2021-04-29 13:30:00-04:00,2021-04-30 10:30:00-04:00,100.0,419.290009,416.929993,41929.000854,41692.999268,-236.001587
3,SPY,2021-05-03 13:30:00-04:00,2021-05-03 15:30:00-04:00,100.0,418.934998,418.179993,41893.499756,41817.999268,-75.500488
4,SPY,2021-05-05 11:30:00-04:00,2021-05-10 13:30:00-04:00,100.0,417.183105,419.769989,41718.310547,41976.998901,258.688354
...,...,...,...,...,...,...,...,...,...
63,SPY,2022-03-25 13:30:00-04:00,2022-03-25 14:30:00-04:00,100.0,451.720001,450.070007,45172.000122,45007.000732,-164.999390
64,SPY,2022-03-25 15:30:00-04:00,2022-03-28 11:30:00-04:00,100.0,452.660004,450.619995,45266.000366,45061.999512,-204.000854
65,SPY,2022-03-28 14:30:00-04:00,2022-03-30 12:30:00-04:00,100.0,454.489990,459.000000,45448.999023,45900.000000,451.000977
66,SPY,2022-04-04 11:30:00-04:00,2022-04-05 12:30:00-04:00,100.0,455.565399,454.184998,45556.539917,45418.499756,-138.040161


In [42]:
#determine a winning vs. losing trade

macd_win = 0
macd_loss = 0
macd_max_win = macd_trade_evaluation_df["Profit/Loss"].max()
macd_max_loss = macd_trade_evaluation_df["Profit/Loss"].min()

for pnl in macd_trade_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        macd_win +=1
    else:
        macd_loss +=1

macd_winrate = macd_win / (macd_win + macd_loss)

print(f"There were {macd_win} winning trades and {macd_loss} losing trades giving us a winrate of {macd_winrate})")
print(f"The largest gain made was ${macd_max_win} and the largest loss was ${macd_max_loss}")


There were 26 winning trades and 42 losing trades giving us a winrate of 0.38235294117647056)
The largest gain made was $1857.000732421875 and the largest loss was $-1120.9991455078125


In [44]:
#export the dataframe to a csv file

#macd_df.to_csv("Resources/macd.csv")

------------

In [ ]:
#alternative macd trading algorithm

macd_a_df = signals_df[["Close", "Open", "High", "Low", "MACD_12_26_9", "MACDh_12_26_9", "MACDs_12_26_9",]].copy()

In [ ]:
# set the share size to 100
share_size = 100

In [ ]:
#Create a new MACD signal

macd_a_df["Signal"]=0.0

macd_a_df["Signal"] = np.where(
    macd_a_df["MACDh_12_26_9"] > 0, 1.0, 0.0
)

#calculate entry/exit points, 1 or -1
macd_a_df["Entry/Exit"] = macd_a_df["Signal"].diff()

#review the dataframe
macd_a_df.tail(10)

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit
Date,,,,,,,,,
2022-04-11 14:30:00-04:00,440.959991,441.529999,443.000000,440.859985,-2.135677,-0.548307,-1.587370,0.0,0.0
2022-04-11 15:30:00-04:00,439.959991,440.950012,441.019989,439.390015,-2.305212,-0.574274,-1.730938,0.0,0.0
2022-04-12 09:30:00-04:00,443.589996,443.079987,445.750000,442.369995,-2.122195,-0.313006,-1.809190,0.0,0.0
2022-04-12 10:30:00-04:00,444.059998,443.600006,444.230011,442.219910,-1.917129,-0.086351,-1.830778,0.0,0.0
2022-04-12 11:30:00-04:00,441.790009,444.070007,444.500000,441.630005,-1.915698,-0.067937,-1.847762,0.0,0.0
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0


In [ ]:
#visualize entry positions relative to close price

entry = macd_a_df[macd_a_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = macd_a_df[macd_a_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the close price for investment
security_close = macd_a_df[['Close']].hvplot(
    line_color = "lightblue",
    ylabel = "Price in $",
    width = 1400,
    height = 600)

entry_exit_plot = security_close * entry * exit
entry_exit_plot.opts(title = "MACD Trading algorithm Entry/Exits")

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [ ]:
#create a column named "Position" by multiplying the share_size by the signal and dividing by 2 (since the signals are '2' and '-2')
# buy a position when the MACD signal = 2 (MACD histogram shows positive value indicating a bullish cross)
# sell a position when the MACD signal = -2 (MACD histogram shows negative value indicating a bearish cross)

macd_a_df["Position"] = share_size * macd_a_df["Signal"]

#review the dataframe
macd_a_df.head()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position
Date,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.647863,-0.549211,1.197073,0.0,NaN,0.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,0.487409,-0.567732,1.055140,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,0.169077,-0.708851,0.877928,0.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,-0.036787,-0.731771,0.694985,0.0,0.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.265860,-0.768676,0.502816,0.0,0.0,0.0


In [ ]:
#find the points in time where a position is purchased or sold
macd_a_df["Entry/Exit Position"] = macd_a_df["Position"].diff()

#review the dataframe
macd_a_df.head()


,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position
Date,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.647863,-0.549211,1.197073,0.0,NaN,0.0,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,0.487409,-0.567732,1.055140,0.0,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,0.169077,-0.708851,0.877928,0.0,0.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,-0.036787,-0.731771,0.694985,0.0,0.0,0.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.265860,-0.768676,0.502816,0.0,0.0,0.0,0.0


In [ ]:
macd_a_df = macd_a_df.dropna()

In [ ]:
#create a Portfolio Holdings column by multiplying the Close price by the Position

macd_a_df["Portfolio Holdings"] = macd_a_df["Close"] * macd_a_df["Position"]

#review the dataframe
macd_a_df.head()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings
Date,,,,,,,,,,,,
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,0.487409,-0.567732,1.055140,0.0,0.0,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,0.169077,-0.708851,0.877928,0.0,0.0,0.0,0.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,-0.036787,-0.731771,0.694985,0.0,0.0,0.0,0.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.265860,-0.768676,0.502816,0.0,0.0,0.0,0.0,0.0
2021-04-20 13:30:00-04:00,411.535004,411.309692,412.100006,411.200012,-0.422760,-0.740461,0.317701,0.0,0.0,0.0,0.0,0.0


In [ ]:
#To calculate Portfolio Cash, subtrace the cumulative sum of the trade cost/proceeds from the initial_capital
#The trade cost proceeds are calculated by multiplying the Close price by the Entry/Exit Position

macd_a_df["Portfolio Cash"] = initial_capital - (macd_a_df["Close"] * macd_a_df["Entry/Exit Position"]).cumsum()

#review the dataframe
macd_a_df.tail(10)


,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash
Date,,,,,,,,,,,,,
2022-04-11 14:30:00-04:00,440.959991,441.529999,443.000000,440.859985,-2.135677,-0.548307,-1.587370,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-11 15:30:00-04:00,439.959991,440.950012,441.019989,439.390015,-2.305212,-0.574274,-1.730938,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 09:30:00-04:00,443.589996,443.079987,445.750000,442.369995,-2.122195,-0.313006,-1.809190,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 10:30:00-04:00,444.059998,443.600006,444.230011,442.219910,-1.917129,-0.086351,-1.830778,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 11:30:00-04:00,441.790009,444.070007,444.500000,441.630005,-1.915698,-0.067937,-1.847762,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568


In [ ]:
#calculate the Portfolo Total by adding Portfolio Cash and Portfolio Holdings
macd_a_df["Portfolio Total"] = macd_a_df["Portfolio Cash"] + macd_a_df["Portfolio Holdings"]

#review the dataframe
macd_a_df.tail()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
Date,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,-2.505660,-0.324251,-2.181409,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568


In [ ]:
#calculate the Portfolio Daily Returns based on the Portfolio Total
macd_a_df["Portfolio Daily Returns"] = macd_a_df["Portfolio Total"].pct_change()

#review the dataframe
macd_a_df.tail()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns
Date,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,-2.505660,-0.324251,-2.181409,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0


In [ ]:
#Calculate the Portfolio Cumulative Returns based on the Portfolio Daily Returns
macd_a_df["Portfolio Cumulative Returns"] = (1 + macd_a_df["Portfolio Daily Returns"]).cumprod() - 1

#review the dataframe
macd_a_df.tail()

,Close,Open,High,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-1.904712,-0.045560,-1.859152,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0,0.022234
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-2.125682,-0.213224,-1.912458,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0,0.022234
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-2.388655,-0.380958,-2.007697,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0,0.022234
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,-2.470944,-0.370598,-2.100347,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0,0.022234
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,-2.505660,-0.324251,-2.181409,0.0,0.0,0.0,0.0,0.0,102223.382568,102223.382568,0.0,0.022234


In [ ]:
#visualize entry positions relative to close price

entry = macd_a_df[macd_a_df["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1000,
    height = 400)

#visualize the exit positions relative to close price
exit = macd_a_df[macd_a_df["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1000,
    height = 400)

#visualize the Portfolio Total for investment
total_portfolio_value = macd_a_df[['Portfolio Total']].hvplot(
    line_color = "lightblue",
    ylabel = "Total Portfolio Value",
    width = 1000,
    height = 400)

portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title = "MACD Trading algorithm - Total Portfolio Value",
    yformatter="%.0f"
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [ ]:
#export the dataframe to a csv file
macd_a_df.to_csv("Resources/macd_alternative.csv")


## Evaluate the portfolio metrics of the MACD algorithm

-------------

In [ ]:
#create a new DataFrame to evaluate the performance of the algorithm

# create a list for the column name

columns = ["Backtest"]

metrics = [

    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio"
    ]
portfolio_evaluation_df = pd.DataFrame(index = metrics, columns=columns)

portfolio_evaluation_df.head()

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [ ]:
# calculate the annualized return

portfolio_evaluation_df.loc["Annualized Return"] = (

    macd_a_df["Portfolio Daily Returns"].mean() * 252

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.003319
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [ ]:
# calculate the cumulative return

portfolio_evaluation_df.loc["Cumulative Returns"] = (

    macd_a_df["Portfolio Cumulative Returns"][-1]
)

portfolio_evaluation_df

,Backtest
Annualized Return,0.003319
Cumulative Returns,0.022234
Annual Volatility,NaN
Sharpe Ratio,NaN


In [ ]:
# Calculate the annual volatility

portfolio_evaluation_df.loc["Annual Volatility"] = (

    macd_a_df["Portfolio Daily Returns"].std() * np.sqrt(252)

)

portfolio_evaluation_df

,Backtest
Annualized Return,0.003319
Cumulative Returns,0.022234
Annual Volatility,0.016386
Sharpe Ratio,NaN


--------------------------

## Create the RSI trading algorithm

In [45]:
#Create a new dataframe for just the RSI signal algorithms

rsi_df = signals_df[["Close", "Open", "High", "Low", "RSI_14", "RSI_Signal"]].copy()

rsi_df.head()

,Close,Open,High,Low,RSI_14,RSI_Signal
Date,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,52.330591,0.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,43.742207,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,33.289577,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,37.216643,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,33.984850,0.0


In [46]:
#check to see how many signals the RSI has given over the past year
rsi_df["RSI_Signal"].value_counts()

 0.0    1501
-1.0     146
 1.0      95
Name: RSI_Signal, dtype: int64

In [47]:
#Implement the RSI trading signal.

# Initialize trade_type column to track buys and sells
rsi_df["side"] = np.nan

# Initialize a cost/proceeds column for recording trade metrics
rsi_df["cost/proceeds"] = np.nan

# Initialize share size and accumulated shares
share_size = 100
short_share_size = -share_size
accumulated_shares = 0

# Initialize variable to hold previous price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a trade at each iteration 
for index, row in rsi_df.iterrows():
    # check market sentement at the period start. We are looking to short if the RSI is showing the market as overbought;  has a value > 70 (indicating a reversal is possible), or go long if it is < 30; indicating market is oversold and a bullish reversal is possible
    # We devide the share size by 2 to establish a beginning position. 
    # All subsequent trades will wind up simultaneously closing the existing position and establish a new position going the other way; so the accumulated_shares position should always be 1/2 of the share size.

    if previous_price == 0:
            rsi_df.loc[index, "side"] = "go long"
            rsi_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
            accumulated_shares += share_size
            rsi_df.loc[index, "trade_volume"] = share_size
            rsi_df.loc[index, "shares"] = accumulated_shares
      
    # Go long if the RSI Signal is greater than 0
    elif row["RSI_Signal"] > 0 and accumulated_shares == 0:
        rsi_df.loc[index, "side"] = "go long"
        rsi_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        rsi_df.loc[index, "trade_volume"] = share_size
        accumulated_shares += share_size
        rsi_df.loc[index, "shares"] = accumulated_shares
        
    # sell short if the RSI signal is less than 0
    elif row["RSI_Signal"] < 0 and accumulated_shares == 100:
        rsi_df.loc[index, "side"] = "liquidate"
        rsi_df.loc[index, "cost/proceeds"] = (row["Close"] * share_size)
        rsi_df.loc[index, "trade_volume"] = share_size
        accumulated_shares += short_share_size
        rsi_df.loc[index, "shares"] = accumulated_shares
    
    # hold if the RSI signal is neutral
    else:
        rsi_df.loc[index, "side"] = "hold"
        rsi_df.loc[index, "shares"] = accumulated_shares

    # update the previous_price to the current row's price
    previous_price = row["Close"]
    
    # if the index is the last index of the DataFrame, close the position
    if index == rsi_df.index[-1]:
        rsi_df.loc[index, "side"] = "liquidate"
        if accumulated_shares < 0:
            rsi_df.loc[index, "cost/proceeds"] = row["Close"] * accumulated_shares
            accumulated_shares = 0; 
            rsi_df.loc[index, "shares"] = accumulated_shares

rsi_df.head()

,Close,Open,High,Low,RSI_14,RSI_Signal,side,cost/proceeds,trade_volume,shares
Date,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,52.330591,0.0,go long,-41523.999023,100.0,100.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,43.742207,0.0,hold,NaN,NaN,100.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,33.289577,0.0,hold,NaN,NaN,100.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,37.216643,0.0,hold,NaN,NaN,100.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,33.984850,0.0,hold,NaN,NaN,100.0


## Visualizing the RSI

In [48]:
# Visualize entry position relative to close price
entry_plot = rsi_df[rsi_df["side"] == "go long"]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit_plot = rsi_df[rsi_df["side"] == "liquidate"]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close_plot = rsi_df[["Close"]].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400
)
# Overlay plots
rsi_entry_exit_plot = entry_plot * exit_plot * security_close_plot
rsi_entry_exit_plot

:Overlay
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)
   .Curve.I          :Curve   [Date]   (Close)

## Backtesting the RSI trading algorithm

In [49]:
rsi_df["value"] =  rsi_df["trade_volume"] * rsi_df["Close"]

In [50]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

rsi_trade_evaluation_df = pd.DataFrame(
     columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss']
)

In [51]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_value = 0.0
exit_portfolio_value = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0


In [52]:
rsi_trade_evaluation_df.head()

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss


In [53]:
# Loop through signal DataFrame
# If `RSI_Signal` is 1, set entry trade metrics
# Else if `RSI_Signal` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in rsi_df.iterrows():
    if row['side'] == "go long" and row["cost/proceeds"] < 0:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['trade_volume']
        entry_share_price = row['Close']

    elif row['side'] == "liquidate" and row["cost/proceeds"] > 0:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['trade_volume'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        rsi_trade_evaluation_df = rsi_trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame


rsi_trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,SPY,2021-04-19 15:30:00-04:00,2021-06-29 09:30:00-04:00,100.0,415.239990,428.179993,41523.999023,42817.999268,1294.000244
1,SPY,2021-07-19 09:30:00-04:00,2021-07-23 09:30:00-04:00,100.0,423.970093,437.558990,42397.009277,43755.899048,1358.889771
2,SPY,2021-08-18 15:30:00-04:00,2021-08-30 09:30:00-04:00,100.0,439.220001,452.165009,43922.000122,45216.500854,1294.500732
3,SPY,2021-09-10 10:30:00-04:00,2021-10-14 11:30:00-04:00,100.0,447.502808,441.894714,44750.280762,44189.471436,-560.809326
4,SPY,2021-11-10 13:30:00-05:00,2021-12-23 09:30:00-05:00,100.0,462.785004,470.630005,46278.500366,47063.000488,784.500122
5,SPY,2022-01-05 14:30:00-05:00,2022-02-01 15:30:00-05:00,100.0,470.295013,452.959991,47029.501343,45295.999146,-1733.502197
6,SPY,2022-02-11 13:30:00-05:00,2022-03-18 14:30:00-04:00,100.0,441.220001,444.029999,44122.000122,44402.999878,280.999756


In [54]:
#let's see how much money our RSI system made!

total_profit_loss = rsi_trade_evaluation_df["Profit/Loss"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $2718.5791015625!


In [55]:
#determine a winning vs. losing trade

rsi_win = 0
rsi_loss = 0
rsi_max_win = rsi_trade_evaluation_df["Profit/Loss"].max()
rsi_max_loss = rsi_trade_evaluation_df["Profit/Loss"].min()

for pnl in rsi_trade_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        rsi_win +=1
    else:
        rsi_loss +=1

rsi_winrate = rsi_win / (rsi_win + rsi_loss)

print(f"There were {rsi_win} winning trades and {rsi_loss} losing trades giving us a winrate of {rsi_winrate})")
print(f"The largest gain made was ${rsi_max_win} and the largest loss was ${rsi_max_loss}")


There were 5 winning trades and 2 losing trades giving us a winrate of 0.7142857142857143)
The largest gain made was $1358.8897705078125 and the largest loss was $-1733.502197265625


In [57]:
#export the dataframe to a csv file
#rsi_df.to_csv("Resources/rsi.csv")

## Create a stochastic algorithm

In [ ]:
# create a new dataframe for the stochastic indicator
stoch_df = signals_df[["Close", "Open", "High", "Low","STOCHk_14_3_3", "STOCHd_14_3_3", "stoch_diff", "Stoch_Signal", "Stoch_Entry/Exit"]].copy()

stoch_df.dropna()
stoch_df.head()

,Close,Open,High,Low,STOCHk_14_3_3,STOCHd_14_3_3,stoch_diff,Stoch_Signal,Stoch_Entry/Exit
Date,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,25.930343,22.103159,3.827184,1.0,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,18.099063,21.674862,-3.575799,-1.0,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,16.004308,20.011238,-4.006930,-1.0,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,11.246037,15.116469,-3.870433,-1.0,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,12.270360,13.173568,-0.903208,-1.0,0.0


In [ ]:
stoch_df["Stoch_Entry/Exit"].value_counts()

 0.0    1327
-2.0     207
 2.0     207
Name: Stoch_Entry/Exit, dtype: int64

In [ ]:
#establish the beginning market sentiment

starting_market_sentement = "unknown"

if stoch_df["stoch_diff"].iloc[0] < 0:
    starting_market_sentement = "bearish"
else:
    starting_market_sentement = "bullish"

print(starting_market_sentement)

bullish


In [ ]:
#Implement the Stochastic trading signal.

# Initialize trade_type column to track buys and sells
stoch_df["side"] = np.nan

# Initialize a cost/proceeds column for recording trade metrics
stoch_df["cost/proceeds"] = np.nan

# Initialize share size and accumulated shares
share_size = 100
short_share_size = -share_size
accumulated_shares = 0

# Initialize variable to hold previous price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a trade at each iteration 
for index, row in stoch_df.iterrows():
    # check market sentement at the period start. We are looking to short if the RSI is showing the market as overbought;  has a value > 70 (indicating a reversal is possible), or go long if it is < 30; indicating market is oversold and a bullish reversal is possible
    # We devide the share size by 2 to establish a beginning position. 
    # All subsequent trades will wind up simultaneously closing the existing position and establish a new position going the other way; so the accumulated_shares position should always be 1/2 of the share size.

           
    #    if starting_market_sentement == "bearish":
    #        stoch_df.loc[index, "side"] = "sell_short"
    #       stoch_df.loc[index, "cost/proceeds"] = (row["Close"] * share_size)
    #        accumulated_shares += short_share_size
    #       stoch_df.loc[index, "shares"] = accumulated_shares
    #    else:
    #        stoch_df.loc[index, "side"] = "go long"
    #        stoch_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
    #        accumulated_shares += share_size
    #        stoch_df.loc[index, "shares"] = accumulated_shares
      
    # Go long to start the time series
    if previous_price == 0:
        stoch_df.loc[index, "side"] = "go long"
        stoch_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        accumulated_shares += share_size
        stoch_df.loc[index, "trade_volume"] = share_size
        stoch_df.loc[index, "shares"] = accumulated_shares
    # Go long if the Stochastic Signal is greater than 0
    elif row["Stoch_Entry/Exit"] > 0:
        stoch_df.loc[index, "side"] = "go long"
        stoch_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        accumulated_shares += share_size
        stoch_df.loc[index, "trade_volume"] = share_size
        stoch_df.loc[index, "shares"] = accumulated_shares
        
    # sell short if the Stochastic signal is less than 0
    elif row["Stoch_Entry/Exit"] < 0:
        stoch_df.loc[index, "side"] = "sell_short"
        stoch_df.loc[index, "cost/proceeds"] = (row["Close"] * share_size)
        accumulated_shares += short_share_size
        stoch_df.loc[index, "trade_volume"] = share_size
        stoch_df.loc[index, "shares"] = accumulated_shares
    
    # hold if the Stochastic signal is neutral
    else:
        stoch_df.loc[index, "side"] = "hold"
        stoch_df.loc[index, "shares"] = accumulated_shares

    # update the previous_price to the current row's price
    previous_price = row["Close"]
    
    # if the index is the last index of the DataFrame, close the position
    if index == stoch_df.index[-1]:
        stoch_df.loc[index, "side"] = "close_position"
        if accumulated_shares < 0:
            stoch_df.loc[index, "cost/proceeds"] = row["Close"] * accumulated_shares
            accumulated_shares = 0; 
            stoch_df.loc[index, "shares"] = accumulated_shares

stoch_df.head()

,Close,Open,High,Low,STOCHk_14_3_3,STOCHd_14_3_3,stoch_diff,Stoch_Signal,Stoch_Entry/Exit,side,cost/proceeds,trade_volume,shares
Date,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,25.930343,22.103159,3.827184,1.0,NaN,go long,-41523.999023,100.0,100.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,18.099063,21.674862,-3.575799,-1.0,-2.0,sell_short,41392.001343,100.0,0.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,16.004308,20.011238,-4.006930,-1.0,0.0,hold,NaN,NaN,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,11.246037,15.116469,-3.870433,-1.0,0.0,hold,NaN,NaN,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,12.270360,13.173568,-0.903208,-1.0,0.0,hold,NaN,NaN,0.0


## Visualize the stochastics

In [ ]:
#graphing positions over time

stoch_df.hvplot(y=["Close", "shares"], kind = "line", figsize = (35,20))

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [ ]:
# Visualize entry position relative to close price
entry = stoch_df[stoch_df["Stoch_Entry/Exit"] == 2.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = stoch_df[stoch_df["Stoch_Entry/Exit"] == -2.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1800,
    height=400
)

# Visualize close price for the investment
security_close = stoch_df[["Close"]].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1800,
    height=400
)

# Visualize moving averages
moving_avgs = macd_df[["MACD_12_26_9", "MACDs_12_26_9"]].hvplot(
    ylabel='Price in $',
    width=1800,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * entry * exit
entry_exit_plot


:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [ ]:
# Visualize moving averages
# Visualize entry position relative to close price
entry = stoch_df[stoch_df["Stoch_Entry/Exit"] == 2.0]["STOCHk_14_3_3"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    width=1800,
    height=400
)

# Visualize exit position relative to close price
exit = stoch_df[stoch_df["Stoch_Entry/Exit"] == -2.0]["STOCHk_14_3_3"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    width=1800,
    height=400
)

stochastics = stoch_df[["STOCHk_14_3_3", "STOCHd_14_3_3"]].hvplot(
    width=1800,
    height=400
)

stochastic_entry_exit = stochastics * entry * exit

stochastic_entry_exit

:Overlay
   .NdOverlay.I              :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.STOCHk_14_3_3.I  :Scatter   [Date]   (STOCHk_14_3_3)
   .Scatter.STOCHk_14_3_3.II :Scatter   [Date]   (STOCHk_14_3_3)

## Backtest the Stochastic Algorithm

In [ ]:
#let's see how much money our Stochastic system made!

total_profit_loss = stoch_df["cost/proceeds"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $-37463.32702636719!


In [ ]:
stoch_df["value"] =  stoch_df["trade_volume"] * stoch_df["Close"]

In [ ]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

stoch_trade_evaluation_df = pd.DataFrame( columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss'])

In [ ]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_value = 0.0
exit_portfolio_value = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0

In [ ]:
# Loop through signal DataFrame
# If `Stoch_Entry/Exit` is 2, set entry trade metrics
# Else if `Stoch_Entry/Exit` is -2, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in stoch_df.iterrows():
    if row['Stoch_Entry/Exit'] == 2:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['trade_volume']
        entry_share_price = row['Close']

    elif row['Stoch_Entry/Exit'] == -2:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['trade_volume'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        stoch_trade_evaluation_df = stoch_trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
stoch_trade_evaluation_df.head()

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,SPY,,2021-04-20 09:30:00-04:00,0.0,0.000000,413.920013,0.000000,41392.001343,41392.001343
1,SPY,2021-04-20 13:30:00-04:00,2021-04-20 14:30:00-04:00,100.0,411.535004,411.904999,41153.500366,41190.499878,36.999512
2,SPY,2021-04-20 15:30:00-04:00,2021-04-22 09:30:00-04:00,100.0,412.239990,415.329987,41223.999023,41532.998657,308.999634
3,SPY,2021-04-22 11:30:00-04:00,2021-04-22 12:30:00-04:00,100.0,416.359985,414.700012,41635.998535,41470.001221,-165.997314
4,SPY,2021-04-23 10:30:00-04:00,2021-04-23 15:30:00-04:00,100.0,416.149994,416.720001,41614.999390,41672.000122,57.000732


In [ ]:
#determine a winning vs. losing trade

stoch_win = 0
stoch_loss = 0
stoch_max_win = macd_trade_evaluation_df["Profit/Loss"].max()
stoch_max_loss = macd_trade_evaluation_df["Profit/Loss"].min()

for pnl in stoch_trade_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        stoch_win +=1
    else:
        stoch_loss +=1

stoch_winrate = stoch_win / (stoch_win + stoch_loss)

print(f"There were {stoch_win} winning trades and {stoch_loss} losing trades giving us a winrate of {stoch_winrate})")
print(f"The largest gain made was ${stoch_max_win} and the largest loss was ${stoch_max_loss}")


There were 89 winning trades and 118 losing trades giving us a winrate of 0.42995169082125606)
The largest gain made was $1857.000732421875 and the largest loss was $-1120.9991455078125


In [ ]:
#export the dataframe to a csv file
stoch_df.to_csv("Resources/stoch.csv")

## Create a custom algorithm
#Signal will need MACD and Stochastic indicators to agree to create a valid trade signal

In [ ]:
#check to see how many custom signals are in the dataframe

signals_df["custom_signal"].value_counts()

 0    1716
-2      16
 2      10
Name: custom_signal, dtype: int64

In [ ]:
#Implement the custom trading signal.

# Initialize trade_type column to track buys and sells
signals_df["side"] = np.nan

# Initialize a cost/proceeds column for recording trade metrics
signals_df["cost/proceeds"] = np.nan

# Initialize share size and accumulated shares
share_size = 100
short_share_size = -share_size
accumulated_shares = 0

# Initialize variable to hold previous price
previous_price = 0

# Loop through the Pandas DataFrame and initiate a trade at each iteration 
for index, row in signals_df.iterrows():
    # check market sentement at the period start. We are looking to short if the RSI is showing the market as overbought;  has a value > 70 (indicating a reversal is possible), or go long if it is < 30; indicating market is oversold and a bullish reversal is possible
    # We devide the share size by 2 to establish a beginning position. 
    # All subsequent trades will wind up simultaneously closing the existing position and establish a new position going the other way; so the accumulated_shares position should always be 1/2 of the share size.

           
    #    if starting_market_sentement == "bearish":
    #        stoch_df.loc[index, "side"] = "sell_short"
    #       stoch_df.loc[index, "cost/proceeds"] = (row["Close"] * share_size)
    #        accumulated_shares += short_share_size
    #       stoch_df.loc[index, "shares"] = accumulated_shares
    #    else:
    #        stoch_df.loc[index, "side"] = "go long"
    #        stoch_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
    #        accumulated_shares += share_size
    #        stoch_df.loc[index, "shares"] = accumulated_shares
      
    # Go long to start the time series
    if previous_price == 0:
        signals_df.loc[index, "side"] = "go long"
        signals_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        accumulated_shares += share_size
        signals_df.loc[index, "trade_volume"] = share_size
        signals_df.loc[index, "shares"] = accumulated_shares
    # Go long if the Stochastic Signal is greater than 0
    elif row["custom_signal"] > 0 and accumulated_shares == 0:
        signals_df.loc[index, "side"] = "go long"
        signals_df.loc[index, "cost/proceeds"] = -(row["Close"] * share_size)
        accumulated_shares += share_size
        signals_df.loc[index, "trade_volume"] = share_size
        signals_df.loc[index, "shares"] = accumulated_shares
        
    # sell short if the Stochastic signal is less than 0
    elif row["custom_signal"] < 0 and accumulated_shares == share_size:
        signals_df.loc[index, "side"] = "liquidate"
        signals_df.loc[index, "cost/proceeds"] = (row["Close"] * share_size)
        accumulated_shares += short_share_size
        signals_df.loc[index, "trade_volume"] = share_size
        signals_df.loc[index, "shares"] = accumulated_shares
    
    # hold if the Stochastic signal is neutral
    else:
        signals_df.loc[index, "side"] = "hold"
        signals_df.loc[index, "shares"] = accumulated_shares

    # update the previous_price to the current row's price
    previous_price = row["Close"]
    
    # if the index is the last index of the DataFrame, close the position
    if index == signals_df.index[-1]:
        signals_df.loc[index, "side"] = "liquidate"
        if accumulated_shares < 0:
            signals_df.loc[index, "cost/proceeds"] = row["Close"] * accumulated_shares
            accumulated_shares = 0; 
            signals_df.loc[index, "shares"] = accumulated_shares

signals_df.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit,side,cost/proceeds,trade_volume,shares
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,...,0.0,1.0,0,NaN,3.827184,NaN,go long,-41523.999023,100.0,100.0
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,...,0.0,-1.0,0,0.0,-3.575799,-2.0,hold,NaN,NaN,100.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,...,0.0,-1.0,0,0.0,-4.006930,0.0,hold,NaN,NaN,100.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,...,0.0,-1.0,0,0.0,-3.870433,0.0,hold,NaN,NaN,100.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,...,0.0,-1.0,0,0.0,-0.903208,0.0,hold,NaN,NaN,100.0


## Visualize the custom signal

In [ ]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["side"] == "go long"]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["side"] == "liquidate"]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1800,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["Close"]].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1800,
    height=400
)

# Visualize moving averages
moving_avgs = macd_df[["MACD_12_26_9", "MACDs_12_26_9"]].hvplot(
    ylabel='Price in $',
    width=1800,
    height=400
)

entry_exit_plot = security_close * entry * exit
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

## Backtest the custom signal

In [ ]:
signals_df["value"] =  signals_df["trade_volume"] * signals_df["Close"]

In [ ]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

custom_evaluation_df = pd.DataFrame( columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss'])

In [ ]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_value = 0.0
exit_portfolio_value = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0

In [ ]:
# Loop through signal DataFrame
# If `RSI_Signal` is 1, set entry trade metrics
# Else if `RSI_Signal` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['side'] == "go long" and row["cost/proceeds"] < 0:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['trade_volume']
        entry_share_price = row['Close']

    elif row['side'] == "liquidate" and row["cost/proceeds"] > 0:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['trade_volume'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        custom_evaluation_df = custom_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame


custom_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,SPY,2021-04-19 15:30:00-04:00,2021-05-28 14:30:00-04:00,100.0,415.239990,420.650085,41523.999023,42065.008545,541.009521
1,SPY,2021-06-04 09:30:00-04:00,2021-06-11 11:30:00-04:00,100.0,421.364990,423.010010,42136.499023,42301.000977,164.501953
2,SPY,2021-10-19 09:30:00-04:00,2021-11-22 15:30:00-05:00,100.0,449.709991,467.529999,44970.999146,46752.999878,1782.000732
3,SPY,2022-03-09 09:30:00-05:00,2022-03-25 10:30:00-04:00,100.0,425.750000,449.070007,42575.000000,44907.000732,2332.000732
4,SPY,2022-03-25 13:30:00-04:00,2022-03-28 11:30:00-04:00,100.0,451.720001,450.619995,45172.000122,45061.999512,-110.000610


In [ ]:
#determine a winning vs. losing trade

custom_win = 0
custom_loss = 0
custom_max_win = custom_evaluation_df["Profit/Loss"].max()
custom_max_loss = custom_evaluation_df["Profit/Loss"].min()

for pnl in custom_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        custom_win +=1
    else:
        custom_loss +=1

custom_winrate = custom_win / (custom_win + custom_loss)

print(f"There were {custom_win} winning trades and {custom_loss} losing trades giving us a winrate of {custom_winrate})")
print(f"The largest gain made was ${custom_max_win} and the largest loss was ${custom_max_loss}")


There were 4 winning trades and 1 losing trades giving us a winrate of 0.8)
The largest gain made was $2332.000732421875 and the largest loss was $-110.0006103515625


In [ ]:
#let's see how much money our custom system made!

total_profit_loss = custom_evaluation_df["Profit/Loss"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $4709.5123291015625!


## Bitcoin data

In [58]:
# Request historic hourly pricing data via finance.yahoo.com API
bitcoin_df = yf.Ticker('BTC-USD').history(period='1y', interval = "1h")[['Close', 'Open', 'High', 'Volume', 'Low']]
bitcoin_df.index.name='Date'
# Review the DataFrame
bitcoin_df.head()

#note the data pulled from the API is already limited to only the hours the market is open

,Close,Open,High,Volume,Low
Date,,,,,
2021-04-13 05:00:00+00:00,60655.421875,60574.828125,60798.707031,0,60496.785156
2021-04-13 06:00:00+00:00,60752.792969,60723.457031,61368.992188,1776222208,60531.351562
2021-04-13 07:00:00+00:00,61096.199219,60878.324219,61232.007812,0,60706.835938
2021-04-13 08:00:00+00:00,62692.648438,61238.320312,62839.523438,5587009536,61158.742188
2021-04-13 09:00:00+00:00,62920.105469,62699.722656,63253.125000,926687232,62387.078125


In [59]:
#add the technical indicators to the bitcoin dataframe, and drop nan values

bitcoin_df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)
bitcoin_df.ta.rsi(close="Close", append=True)
bitcoin_df.ta.stoch(close="Close", append=True)
bitcoin_df = bitcoin_df.dropna()

bitcoin_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3
Date,,,,,,,,,,,
2021-04-14 15:00:00+00:00,63200.328125,63143.218750,63459.949219,560734208,62603.304688,352.438978,-227.296078,579.735056,47.549614,30.005740,45.569075
2021-04-14 16:00:00+00:00,63636.906250,63171.070312,63826.519531,0,62987.664062,316.196716,-210.830672,527.027388,52.571302,29.383467,33.556675
2021-04-14 17:00:00+00:00,62808.371094,63655.718750,63854.359375,0,62728.757812,218.104428,-247.138368,465.242796,43.967897,27.077575,28.822261
2021-04-14 18:00:00+00:00,62198.378906,62545.937500,62730.660156,623951872,61868.546875,90.105629,-300.109733,390.215362,38.918127,21.942582,26.134541
2021-04-14 19:00:00+00:00,62244.957031,62217.371094,62557.453125,1172676608,61554.796875,-7.489541,-318.163923,310.674382,39.489616,13.650148,20.890102


In [61]:
# Initialize the new Signal columns
bitcoin_df['MACD_Signal'] = 0.0
bitcoin_df["RSI_Signal"] = 0.0
bitcoin_df["Stoch_Signal"]= 0.0
bitcoin_df["custom_signal"] = 0.0

#MACD Logic:
# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
bitcoin_df.loc[(bitcoin_df['MACDh_12_26_9'] >= 0), 'MACD_Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
bitcoin_df.loc[(bitcoin_df['MACDh_12_26_9'] < 0), 'MACD_Signal'] = -1

bitcoin_df["MACD_Entry/Exit"] = bitcoin_df["MACD_Signal"].diff()



#RSI Logic:
# When RSI < 30, buy signal are authorized
bitcoin_df.loc[(bitcoin_df['RSI_14'] < 30), 'RSI_Signal'] = 1

#When RSI > 70 sell signals are authorized
bitcoin_df.loc[(bitcoin_df['RSI_14'] > 70), 'RSI_Signal'] = -1

#when  30 < RSI < 70, neutral signal, do not enter new trades. defult value of 0 will suffice.





#Stochastic Logic:
#calculate the difference between the "k" and "d" stochastics
bitcoin_df["stoch_diff"] = bitcoin_df["STOCHk_14_3_3"] - bitcoin_df["STOCHd_14_3_3"]

#when the "k" crosses below the "d", that is a bearish signal.
bitcoin_df.loc[bitcoin_df["stoch_diff"] < 0, "Stoch_Signal"] = -1

#when the "k" crosses above the "d", that is a bullish signal.
bitcoin_df.loc[bitcoin_df["stoch_diff"]  > 0, "Stoch_Signal"] = 1

bitcoin_df["Stoch_Entry/Exit"] = bitcoin_df["Stoch_Signal"].diff()

bitcoin_df.head(50)

#custom logic:
#combine valid MACD and Stochastic signals to only enter a trade when multiple indicators are in agreement.
bitcoin_df["custom_signal"]=0

#def custom_indicator (indicator_1. indicator_2)
for index, row in bitcoin_df.iterrows():

    if (row["Stoch_Entry/Exit"] == 2 and row["MACD_Entry/Exit"] == 2):
        bitcoin_df.at[index, "custom_signal"] = 2
    elif (row["Stoch_Entry/Exit"] == -2 and row["MACD_Entry/Exit"] == -2):
        bitcoin_df.at[index, "custom_signal"] = -2
    else:
        bitcoin_df.at[index, "custom_signal"] = 0    


bitcoin_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-14 15:00:00+00:00,63200.328125,63143.218750,63459.949219,560734208,62603.304688,352.438978,-227.296078,579.735056,47.549614,30.005740,45.569075,-1.0,0.0,-1.0,0,NaN,-15.563335,NaN
2021-04-14 16:00:00+00:00,63636.906250,63171.070312,63826.519531,0,62987.664062,316.196716,-210.830672,527.027388,52.571302,29.383467,33.556675,-1.0,0.0,-1.0,0,0.0,-4.173208,0.0
2021-04-14 17:00:00+00:00,62808.371094,63655.718750,63854.359375,0,62728.757812,218.104428,-247.138368,465.242796,43.967897,27.077575,28.822261,-1.0,0.0,-1.0,0,0.0,-1.744686,0.0
2021-04-14 18:00:00+00:00,62198.378906,62545.937500,62730.660156,623951872,61868.546875,90.105629,-300.109733,390.215362,38.918127,21.942582,26.134541,-1.0,0.0,-1.0,0,0.0,-4.191959,0.0
2021-04-14 19:00:00+00:00,62244.957031,62217.371094,62557.453125,1172676608,61554.796875,-7.489541,-318.163923,310.674382,39.489616,13.650148,20.890102,-1.0,0.0,-1.0,0,0.0,-7.239954,0.0


In [63]:
#export the dataframe to a csv file
bitcoin_df.to_csv("../Resources/bitcoin.csv")